In [1]:
from datasets import load_dataset, load_from_disk
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
)
import numpy as np
import os

model_name = "bert-base-cased"
dataset_folder = "saved_datasets"
dataset_name = "ai4privacy/pii-masking-400k"
dataset_path = os.path.join(dataset_folder, dataset_name.replace("/", "_"))

os.makedirs(dataset_folder, exist_ok=True)

if os.path.exists(dataset_path):
    dataset = load_from_disk(dataset_path)
else:
    dataset = load_dataset(dataset_name)
    dataset.save_to_disk(dataset_path)

unique_labels = set()
for row in dataset["train"]["mbert_token_classes"]:
    unique_labels.update(row)
label_list = sorted(unique_labels)
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}
num_labels = len(label_list)

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["mbert_tokens"], is_split_into_words=True, truncation=True
    )
    labels = []
    for i, label in enumerate(examples["mbert_token_classes"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        prev_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != prev_word_id:
                label_ids.append(label_to_id[label[word_id]])
            else:
                label_ids.append(label_to_id[label[word_id]])
            prev_word_id = word_id
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs



c:\Users\bhatt\anaconda3\envs\pii_eraser\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

batch_size = 8

# Take 10% of the training data
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.1 * len(tokenized_datasets["train"]))))

args = TrainingArguments(
    output_dir="./pii_model",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("./pii_model")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'max_train_samples'